In [1]:
import pandas as pd
import numpy as np
import os
from datetime import time,timedelta
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from scipy.stats import spearmanr, pearsonr


In [2]:
head_dir = './data'
# days示例：['1','2']
# stock示例：'A'
def generate_and_process_data(days:list[str],stock:str,head_dir:str = head_dir):
    
    data_daylist = []
    days = sorted(days)

    for day in days:
        df = pd.read_csv(f'{head_dir}/{day}/{stock}.csv') #读取

        df['LastPrice'] = df['LastPrice'].replace(0,np.nan).ffill() #将零值用前一个非零值填充
        df = df.rename(columns={'BidVolum5':'BidVolume5',
                                'AskVolum5':'AskVolume5'}) #修改了几个出错的列名，看着难受
        data_daylist.append(df)
        
    return data_daylist
    



In [5]:
A_list = generate_and_process_data(['1'],'A')
A_df = A_list[0]

print(A_df['Time'].dtype)

def exist_exception(mask:pd.Series):
    mark = True
    for i in mask:
        if not i:
            print('存在异常值')
            mark = False
            break
    if mark:
        print('不存在异常值')

exist_exception(A_df['AskPrice1'] > A_df['BidPrice1'])
exist_exception((A_df['LastPrice'] >= A_df['BidPrice1']) & (A_df['LastPrice'] <= A_df['AskPrice1'])) #由于之前的前向填充处理，这里确实会出现异常值
exist_exception(A_df['LastPrice'] != 0)


int64
不存在异常值
存在异常值
不存在异常值


In [6]:
all_days = ['1','2','3','4','5']

# list是以天为单位的列表，每个元素是一个dataframe
A_list = generate_and_process_data(all_days,'A')
B_list = generate_and_process_data(all_days,'B')
C_list = generate_and_process_data(all_days,'C')
D_list = generate_and_process_data(all_days,'D')
E_list = generate_and_process_data(all_days,'E')

# 由于一天的刚开始订单量异常多，故分开考虑开盘时5分钟前后的数据
A_list_5min = [df[df['Time'] < 93500000] for df in A_list]
B_list_5min = [df[df['Time'] < 93500000] for df in B_list]
C_list_5min = [df[df['Time'] < 93500000] for df in C_list]
D_list_5min = [df[df['Time'] < 93500000] for df in D_list]
E_list_5min = [df[df['Time'] < 93500000] for df in E_list]

A_list_rest = [df[df['Time'] >= 93500000] for df in A_list]
B_list_rest = [df[df['Time'] >= 93500000] for df in B_list]
C_list_rest = [df[df['Time'] >= 93500000] for df in C_list]
D_list_rest = [df[df['Time'] >= 93500000] for df in D_list]
E_list_rest = [df[df['Time'] >= 93500000] for df in E_list]

# 将5天的数据拼到一起，形成一个dataframe
A_df_5min_all = pd.concat(A_list_5min)
B_df_5min_all = pd.concat(B_list_5min)
C_df_5min_all = pd.concat(C_list_5min)
D_df_5min_all = pd.concat(D_list_5min)
E_df_5min_all = pd.concat(E_list_5min)

A_df_rest_all = pd.concat(A_list_rest)
B_df_rest_all = pd.concat(B_list_rest)
C_df_rest_all = pd.concat(C_list_rest)
D_df_rest_all = pd.concat(D_list_rest)
E_df_rest_all = pd.concat(E_list_rest)


滚动标准化可能带来的问题是，因子会受极端值影响，从而可能整体失真，AI建议我们用mad法，去除极端值，可以一试

这是一个非常深刻且切中要害的问题。您完全正确，滚动标准化虽然必要，但确实会带来“受极端值影响而导致因子值失真”的风险。这正是在实盘因子工程中必须严肃对待和妥善处理的核心问题之一。
下面我们来详细拆解这个问题，并提供解决方案。
为什么滚动标准化会受极端值影响？
想象一个简单的滚动Z-Score标准化公式：
因子值_标准化 = (因子值_t - 滚动窗口内因子均值) / 滚动窗口内因子标准差
在某个时刻t，一个巨大的异常值（极端值）进入了滚动窗口。
影响1：拉偏均值：这个异常值会显著拉高或拉低窗口内的均值。
影响2：炸大标准差：由于标准差衡量的是波动，这个异常值会使得窗口内的标准差急剧增大。
双重打击：在接下来的一个窗口期内，所有正常因子值的标准化计算都会基于这个被扭曲的均值和被放大的标准差。导致：
原本正常的因子值会被压缩到一个非常小的区间内（例如，全部被压缩到[-0.5, 0.5]），失去了区分度。
标准化后的序列会出现一个明显的“凹陷”或“平台”，这完全是由单个数据错误引起的，而非市场的真实变化。
这种情况在高频数据中尤其致命，比如一笔“乌龙指”交易或者一个数据错误，就可能导致几分钟内的因子失效。

解决方案：从“均值-标准差”到“稳健标准化”
解决方案的核心是：在滚动标准化之前或之中，使用对异常值不敏感的“稳健统计量”来替代均值和标准差。
以下是几种行之有效的方法，按推荐顺序排列：

方法一：在标准化前先处理极端值（去极值）
这是最常用、最有效的流程。先对原始因子序列进行去极值处理，然后再进行滚动标准化。
MAD法（中位数绝对偏差法）
思想：用中位数代替均值，用绝对偏差代替标准差。这两个统计量对异常值都不敏感。
步骤：
计算滚动窗口内因子值的中位数 MEDIAN。
计算每个因子值与中位数的绝对偏差：|因子值 - MEDIAN|。
计算这些绝对偏差的中位数，得到 MAD。
找到一个稳健的“标准差异常值”判断阈值，通常为：MEDIAN ± k * MAD（常见的k取3或5）。
将超过阈值的因子值截断（Winsorization）或设为中位数。
优势：非常稳健，即使窗口内存在多个异常值也能较好处理。
分位数法
思想：直接使用因子分布的分位数来判定异常值。
步骤：将因子值限制在滚动窗口的某个分位数区间内，例如 [1%分位数, 99%分位数] 或 [5%分位数, 95%分位数]。超出部分用分位数值代替。
代码示例（MAD去极值 + 滚动标准化）：
def robust_rolling_zscore(series, window, k=5):
    """
    稳健的滚动标准化：先MAD去极值，再Z-Score
    """
    # 1. 计算滚动中位数和MAD
    rolling_median = series.rolling(window).median()
    rolling_mad = (series - rolling_median).abs().rolling(window).median()
    
    # 2. 计算上下限，并进行去极值（Winsorization）
    upper_bound = rolling_median + k * rolling_mad
    lower_bound = rolling_median - k * rolling_mad
    # 将超出界限的值用界限值替代
    winsorized_series = series.clip(lower=lower_bound, upper=upper_bound)
    
    # 3. 对去极值后的序列进行标准的滚动Z-Score标准化
    # 使用去极值后序列的滚动均值和标准差
    rolling_mean_win = winsorized_series.rolling(window).mean()
    rolling_std_win = winsorized_series.rolling(window).std()
    
    return (winsorized_series - rolling_mean_win) / rolling_std_win
    
方法二：直接使用稳健的标准化方法
不先处理原始值，而是在标准化公式中直接使用稳健统计量。
基于中位数和MAD的标准化
公式：因子值_标准化 = (因子值_t - 滚动中位数) / (滚动MAD)
注意：为了使“MAD”与“标准差”在尺度上可比，通常会将MAD乘以一个常数（约1.4826），因为正态分布中 标准差 ≈ 1.4826 * MAD。
修正公式：因子值_标准化 = (因子值_t - 滚动中位数) / (滚动MAD * 1.4826)
方法三：使用次序统计量进行标准化
思想：不依赖具体的数值分布，而是基于排名。
方法：在滚动窗口内，将因子值转换为其在窗口内的分位数排名或正态化分位数。例如，使用scipy.stats.rankdata或pandas.DataFrame.rank计算出每个点的分位数（介于0和1之间），然后再映射到标准正态分布的分位数上。这种方法完全不受异常值影响，但可能会损失一些数值间的线性关系。
实战建议与工作流
可视化！可视化！可视化！
将原始因子序列、滚动均值/标准差序列、以及标准化后的因子序列同时画出来。
异常值导致的失真在图上会非常明显（比如标准差突然出现一个尖峰，随后因子被压缩）。
建立标准化流水线：
首选：方法一，即“去极值 -> 标准化”的流水线。这是业界的黄金标准。
参数选择：对于去极值的参数（如MAD的k值），需要结合因子的历史分布进行回测确定。通常k=3对应较严格的标准，k=5更宽松。
效果评估：
一个稳健的标准化方法，其输出的因子序列应该具有相对稳定的波动性（例如，滚动标准差不会出现剧烈的、短暂的跳跃）。
最终，还是要看经过稳健处理后的因子在样本外测试中的IC序列是否更稳定，IR（信息比率）是否得到提升。
总结
方法
	
优点
	
缺点
	
适用场景


传统Z-Score​
	
计算简单，意义直观
	
对异常值极度敏感
	
数据非常干净，或仅用于探索性分析


MAD去极值+Z-Score​
	
非常稳健，效果好，业界主流​
	
计算稍复杂，需选择参数k
	
绝大多数场景，强烈推荐​


中位数+MAD标准化​
	
无需去极值步骤，一步到位
	
对极端密集分布可能稍敏感
	
对实时性要求极高，可接受轻微失真


分位数标准化​
	
完全不受异常值影响，结果稳定
	
损失数值间的部分信息，计算量大
	
因子值与收益率为单调但非线性的关系
结论：您的担忧是完全必要的。​ 直接使用传统的滚动Z-Score标准化是危险的。请务必在标准化前加入稳健的去极值处理（如MAD法），这样可以极大程度地免疫于极端值的影响，保证因子在生产环境中的稳定性和可靠性。

In [7]:
#滚动标准化：
def rolling_zscore(series:pd.Series,
                   window_z:int,
):
    
    series_mean = series.rolling(window_z,min_periods=1).mean().shift(1)
    series_std = series.rolling(window_z,min_periods=1).std().shift(1)+1e-8
    series_std = series_std.replace(0,np.nan).fillna(method="bfill")
    series_zscore = (series-series_mean)/series_std
    return series_zscore

def rolling_minmax(series:pd.Series,
                   window_z:int
):

    series_min = series.rolling(window_z,min_periods=1).min().shift(1)
    series_max = series.rolling(window_z,min_periods=1).max().shift(1)
    series_minmax = (series-series_min)/(series_max+series_min+1e-8)
    return series_minmax

# 稳健的滚动标准化：
def mad_rolling_zscore(series:pd.Series,
                       window:int,
                       k=5):
    """
    稳健的滚动标准化：先MAD去极值，再Z-Score
    """
    # 1. 计算滚动中位数和MAD
    rolling_median = series.rolling(window,min_periods=1).median()
    rolling_mad = (series - rolling_median).abs().rolling(window,min_periods=1).median()
    
    # 2. 计算上下限，并进行去极值（Winsorization）
    upper_bound = rolling_median + k * rolling_mad
    lower_bound = rolling_median - k * rolling_mad
    # 将超出界限的值用界限值替代
    winsorized_series = series.clip(lower=lower_bound, upper=upper_bound)
    
    # 3. 对去极值后的序列进行标准的滚动Z-Score标准化
    # 使用去极值后序列的滚动均值和标准差
    rolling_mean_win = winsorized_series.rolling(window,min_periods=1).mean()
    rolling_std_win = winsorized_series.rolling(window,min_periods=1).std()
    
    return (winsorized_series - rolling_mean_win) / rolling_std_win


In [ ]:
# 回溯区间内的平均价格和回溯区间起始价格之间计算一个收益率
def cal_PastReturn(lastprice:pd.Series,
                   window:int):
    base = lastprice.shift(window).fillna(method='bfill')
    ave = lastprice.rolling(window,min_periods=1).mean()
    ret = ave/base - 1
    return ret

# 回溯区间内成交价的高低价差和当前成交价的比值
def cal_HLR(lastprice:pd.Series,
                   window:int):
    high = lastprice.rolling(window,min_periods=1).max()
    low = lastprice.rolling(window,min_periods=1).min()
    hlr = (high - low)/lastprice

    return hlr

# 回溯区间买卖价差的标准差
def cal_BSV(bidprice:pd.Series,
            askprice:pd.Series,
            window:int):
    minus = bidprice-askprice
    bsv = minus.rolling(window,min_periods=1).std()
    return bsv

# 下行收益率的波动率：只考虑相对前一刻下跌的tick的跌幅,计算其标准差
def cal_DownLogreturn_std(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    downlogreturn = logreturn.where(logreturn < 0,np.nan).rolling(window,min_periods=1).std()

    return downlogreturn

# 下行收益率的均值
def cal_DownLogreturn(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    downlogreturn = logreturn.where(logreturn < 0,np.nan).rolling(window,min_periods=1).mean()

    return downlogreturn

# 上行收益率的波动率：只考虑相对前一刻上涨的tick的跌幅,计算其标准差
def cal_UpLogreturn_std(lastprice:pd.Series,
                      window:int):
    logreturn = np.log(lastprice).diff()
    uplogreturn = logreturn.where(logreturn > 0,np.nan).rolling(window,min_periods=1).std()

    return uplogreturn

# 上行收益率的均值
def cal_UpLogreturn(lastprice:pd.Series,
                    window:int):
    logreturn = np.log(lastprice).diff()
    uplogreturn = logreturn.where(logreturn > 0,np.nan).rolling(window,min_periods=1).mean()

    return uplogreturn

# 对成交价进行时间加权平均（ewm指数加权，越近的时间权重越大)后，长时间窗口和短时间窗口的差
def cal_DIF(lastprice:pd.Series,
            window_long:int,
            window_short:int):
    long = lastprice.ewm(span=window_long,adjust=False).mean()
    short = lastprice.ewm(span=window_short,adjust=False).mean()
    dif = long - short

    return dif

# 对dif进行平均
def cal_DEA(lastprice:pd.Series,
            window_long:int,
            window_short:int,
            window_dea:int):
    
    dea = cal_DIF(lastprice,window_long,window_short).rolling(window_dea,min_periods=1).mean()

    return dea

# 滚动成交量加权均价（不用lastprice而是amount和volume的比值）
def cal_VWAP(sellamount:pd.Series,
             buyamount:pd.Series,
             sellvolume:pd.Series,
             buyvolume:pd.Series,
             window:int):
    sumamount = sellamount + buyamount
    sumvolume = sellvolume + buyvolume
    vwap = sumamount.rolling(window,min_periods=1).sum() / sumvolume.rolling(window,min_periods=1).sum()

    return vwap

# 差分后用ewm算涨跌比值，不用RSI
def cal_RS(lastprice:pd.Series,
            window:int):
    minus = lastprice.diff()
    gain = minus.where(minus > 0,0).ewm(span=window,adjust=False).mean()
    loss = -minus.where(minus < 0,0).ewm(span=window,adjust=False).mean()
    rs = gain / (loss + 1e-8)

    return rs

# AI推荐的RSI的差分，或许可以衡量动能
def cal_RSImomentum(lastprice:pd.Series,
                    window:int):
    rs = cal_RS(lastprice,window)
    rsi = 100 - 100/(1 + rs)
    rsi_momentum = rsi.diff().bfill().rolling(window,min_periods=1).mean()

    return rsi_momentum

# 无意间发现的因子，直接对lastprice用ewm算涨幅的加权平均，或许可以衡量动能
def cal_EWP(lastprice:pd.Series,
            window:int):
    gain = lastprice.where(lastprice > 0,0).ewm(span=window,adjust=False).mean()

    return gain

# ?
def cal_EWP_reciprocalmomentum(lastprice:pd.Series,
                     window:int):
    gain = cal_EWP(lastprice,window)
    gain_reciprocal = 1/(gain + 1e-8) #取倒数，或许可以衡量反转动能
    ewp_momentum = gain_reciprocal.diff().bfill().rolling(window,min_periods=1).mean()

    return ewp_momentum

# 经过修正的RSI因子
def cal_RSI(lastprice:pd.Series,
            window:int):
    rs = cal_RS(lastprice,window)
    rsi = 50 - 100/(1 + rs)
    
    return rsi
#计算VOI和OIR所用的五档权重函数
def weigh(
        df:pd.DataFrame,
        buy_list:list=["BidVolume1","BidVolume2","BidVolume3","BidVolume4","BidVolume5"],
        sell_list:list=["AskVolume1","AskVolume2","AskVolume3","AskVolume4","AskVolume5"],
        switch_buyorsell:str="s" ,
        decay:int = 0.5   
        ):
    data = df.copy()
    data_sum = pd.Series(0,index=data.index)
    if switch_buyorsell == "s":
        for i,index in enumerate(sell_list):
            data_1 = data[index] * np.exp(-decay*i)
            data_sum += data_1
    elif switch_buyorsell == "b":
        for i,index in enumerate(buy_list):
            data_1 = data[index] * np.exp(-decay*i)
            data_sum += data_1
    return data_sum/ np.sum([np.exp(-decay * i) for i in range(5)]) #归一化

''' 计算订单失衡情况，加权后的五档买量和五档卖量的差
def cal_OBI(df:pd.DataFrame,
            window:int):
    buyorder = weigh(df,switch_buyorsell="b").rolling(window,min_periods=1).mean()
    sellorder = weigh(df,switch_buyorsell="s").rolling(window,min_periods=1).mean()
    obi = buyorder - sellorder
    
    return obi
但是经过IC测试，发现这个指标的表现很差，可能是计算方式不对，或者这个指标本身就不太有用，暂时先放弃了
'''
# 换一种计算订单失衡情况的方式，不用五档量，而用挂单买量和卖量的差
def cal_OBI(orderbuy:pd.Series,
            ordersell:pd.Series,
            window:int):
    obi = (orderbuy - ordersell).rolling(window,min_periods=1).mean()

    return obi

# 订单失衡比（同样换一种方式）
def cal_OIR(orderbuy:pd.Series,
            ordersell:pd.Series,
            window:int):
    buyorder = orderbuy.rolling(window,min_periods=1).mean()
    sellorder = ordersell.rolling(window,min_periods=1).mean()
    oir = (buyorder - sellorder) / (buyorder + sellorder + 1e-8)

    return oir

# 主动买入和卖出量之差
def cal_VOI(orderbuyvolume:pd.Series,
            ordersellvolume:pd.Series,
            window:int):
    voi = (orderbuyvolume - ordersellvolume).rolling(window,min_periods=1).mean()

    return voi

In [9]:
# 计算平均订单量失衡情况：平均买单量和平均卖单量的差
def cal_AVO(orderbuyvolume:pd.Series,
            ordersellvolume:pd.Series,
            orderbuynum:pd.Series,
            ordersellnum:pd.Series,
            window:int):
    
    avbuy = orderbuyvolume / (orderbuynum + 1e-8)
    avsell = ordersellvolume / (ordersellnum + 1e-8)
    avo = (avbuy - avsell).rolling(window,min_periods=1).mean()
    return avo

# 订单失衡和买卖价差的结合指标：订单失衡乘以买卖价差的标准差
def cal_BSVmultiOBI(orderbuy:pd.Series,
            ordersell:pd.Series,
            bidprice:pd.Series,
            askprice:pd.Series,
            window:int):
    obi = cal_OBI(orderbuy,ordersell,window)
    bsv = cal_BSV(bidprice,askprice,window)
    bsvmultiobi = bsv * obi
    return bsvmultiobi

# 上行收益率和下行收益率的结合指标：二者的差值
def cal_UpDownLogreturn(lastprice:pd.Series,
                        window:int):
    downlogreturn = cal_DownLogreturn(lastprice,window)
    uplogreturn = cal_UpLogreturn(lastprice,window)
    updownlogreturn = uplogreturn + downlogreturn
    return updownlogreturn

In [17]:
def calculate_ic_series(factor_series: pd.Series, 
                       return_series: pd.Series, 
                       method: str = 'spearman', # 默认斯皮尔曼相关系数
                       min_periods: int = 30) -> pd.Series:
    """
    计算因子值与未来收益率之间的IC序列
    
    参数:
    ----------
    factor_series : pd.Series
        因子值序列，索引应为时间戳
    return_series : pd.Series
        未来收益率序列，索引应与factor_series一致
    method : str, 可选 ('spearman', 'pearson')
        相关系数计算方法:
        - 'spearman': 斯皮尔曼秩相关系数（推荐，对异常值不敏感）
        - 'pearson': 皮尔逊相关系数（要求数据近似正态分布）
    min_periods : int
        计算相关系数所需的最小样本数
    
    返回:
    -------
    pd.Series
        IC值序列，索引与输入序列一致
    """
    
    # 检查输入数据
    if not isinstance(factor_series, pd.Series) or not isinstance(return_series, pd.Series):
        raise TypeError("输入必须是pandas Series")
    
    if len(factor_series) != len(return_series):
        raise ValueError("因子序列和收益率序列长度必须一致")
    
    # 对齐索引
    aligned_data = pd.DataFrame({
        'factor': factor_series,
        'return': return_series
    }).dropna()
    
    if len(aligned_data) == 0:
        raise ValueError("对齐后的数据为空，请检查输入序列")
    
    # 根据方法选择计算函数
    if method == 'spearman':
        def corr_func(x, y):
            return spearmanr(x, y).correlation
    elif method == 'pearson':
        def corr_func(x, y):
            return pearsonr(x, y)[0]
    else:
        raise ValueError("method参数必须是'spearman'或'pearson'")
    
    # 计算IC序列
    ic_values = []
    dates = []
    
    # 按时间顺序计算滚动IC
    for i in range(min_periods, len(aligned_data)):
        current_data = aligned_data.iloc[:i+1]
        
        # 确保有足够的数据点
        if len(current_data) >= min_periods:
            try:
                ic = corr_func(current_data['factor'].values, 
                              current_data['return'].values)
                ic_values.append(ic)
                dates.append(aligned_data.index[i])
            except (ValueError, TypeError):
                # 处理计算错误（如所有值相同的情况）
                ic_values.append(np.nan)
                dates.append(aligned_data.index[i])
    
    return pd.Series(ic_values, index=dates, name=f'IC_{method}')

# 改进版本的IC序列计算，使用滚动窗口计算IC，避免每次都使用从开始到当前的全部数据，减少计算量并更关注近期表现
def cal_rolling_ic_series(factor_series: pd.Series, 
                       return_series: pd.Series, 
                       method: str = 'spearman', # 默认斯皮尔曼相关系数
                       min_periods: int = 7200) -> pd.Series:
    """
    计算因子值与未来收益率之间的IC序列
    
    参数:
    ----------
    factor_series : pd.Series
        因子值序列，索引应为时间戳
    return_series : pd.Series
        未来收益率序列，索引应与factor_series一致
    method : str, 可选 ('spearman', 'pearson')
        相关系数计算方法:
        - 'spearman': 斯皮尔曼秩相关系数（推荐，对异常值不敏感）
        - 'pearson': 皮尔逊相关系数（要求数据近似正态分布）
    min_periods : int
        计算相关系数所需的最小样本数
    
    返回:
    -------
    pd.Series
        IC值序列，索引与输入序列一致
    """
    
    # 检查输入数据
    if not isinstance(factor_series, pd.Series) or not isinstance(return_series, pd.Series):
        raise TypeError("输入必须是pandas Series")
    
    if len(factor_series) != len(return_series):
        raise ValueError("因子序列和收益率序列长度必须一致")
    
    # 对齐索引
    aligned_data = pd.DataFrame({
        'factor': factor_series,
        'return': return_series
    }).dropna()
    
    if len(aligned_data) == 0:
        raise ValueError("对齐后的数据为空，请检查输入序列")
    
    # 根据方法选择计算函数
    if method == 'spearman':
        def corr_func(x, y):
            return spearmanr(x, y).correlation
    elif method == 'pearson':
        def corr_func(x, y):
            return pearsonr(x, y)[0]
    else:
        raise ValueError("method参数必须是'spearman'或'pearson'")
    
    # 计算IC序列
    ic_values = []
    dates = []
    
    # 按时间顺序计算滚动IC
    for i in range(min_periods, len(aligned_data)):
        current_data = aligned_data.iloc[i-min_periods:i]
        
        # 确保有足够的数据点
        if len(current_data) >= min_periods:
            try:
                ic = corr_func(current_data['factor'].values, 
                              current_data['return'].values)
                ic_values.append(ic)
                dates.append(aligned_data.index[i])
            except (ValueError, TypeError):
                # 处理计算错误（如所有值相同的情况）
                ic_values.append(np.nan)
                dates.append(aligned_data.index[i])
    
    return pd.Series(ic_values, index=dates, name=f'IC_{method}')

# 只计算一个IC统计指标的函数，方便快速评估因子表现
def cal_ic_mean(factor_series: pd.Series,
                      return_series: pd.Series,
                      method: str = 'spearman',) -> float:
    # 检查输入数据
    if not isinstance(factor_series, pd.Series) or not isinstance(return_series, pd.Series):
        raise TypeError("输入必须是pandas Series")
    
    if len(factor_series) != len(return_series):
        raise ValueError("因子序列和收益率序列长度必须一致")
    
    # 对齐索引
    aligned_data = pd.DataFrame({
        'factor': factor_series,
        'return': return_series
    }).dropna()
    
    if len(aligned_data) == 0:
        raise ValueError("对齐后的数据为空，请检查输入序列")
    
    # 根据方法选择计算函数
    if method == 'spearman':
        def corr_func(x, y):
            return spearmanr(x, y).correlation
    elif method == 'pearson':
        def corr_func(x, y):
            return pearsonr(x, y)[0]
    else:
        raise ValueError("method参数必须是'spearman'或'pearson'")
    
    return corr_func(aligned_data['factor'].values, aligned_data['return'].values)

def analyze_ic_series(ic_series: pd.Series) -> dict:
    """
    对IC序列进行统计分析
    
    参数:
    ----------
    ic_series : pd.Series
        calculate_ic_series函数返回的IC序列
    
    返回:
    -------
    dict
        包含IC统计指标的字典
    """
    ic_clean = ic_series.dropna()
    
    if len(ic_clean) == 0:
        return {"error": "无有效IC值"}
    
    stats = {
        'IC_mean': ic_clean.mean(),           # IC均值
        'IC_std': ic_clean.std(),             # IC标准差
        'IC_IR': ic_clean.mean() / ic_clean.std() if ic_clean.std() != 0 else 0,  # 信息比率
        'IC_positive_ratio': (ic_clean > 0).mean(),  # IC为正的比例
        'IC_significant_ratio': (ic_clean.abs() > 0.02).mean(),  # |IC| > 0.02的比例
        'IC_abs_mean': ic_clean.abs().mean(),  # IC绝对值均值
        'observation_count': len(ic_clean),   # 有效观测数
        'total_periods': len(ic_series)       # 总期数
    }
    
    return stats

In [14]:
E_lastprice_rest_all = E_df_rest_all['LastPrice']
target_rest = E_df_rest_all['Return5min']

In [ ]:

bsv_factor =  cal_BSV(E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window = 600)
bsv_ic = calculate_ic_series(bsv_factor,target_rest)

print(analyze_ic_series(bsv_ic))

{'IC_mean': np.float64(-0.09093616075893289), 'IC_std': 0.11202381942780285, 'IC_IR': np.float64(-0.8117573675261043), 'IC_positive_ratio': np.float64(0.00584557025797561), 'IC_significant_ratio': np.float64(0.8611732629987998), 'IC_abs_mean': np.float64(0.09193411682354057), 'observation_count': 134974, 'total_periods': 134974}


In [15]:
bsv_factor = cal_BSV(E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window = 600)
bsv_ic = cal_rolling_ic_series(bsv_factor,target_rest)
print(analyze_ic_series(bsv_ic))

{'IC_mean': np.float64(-0.11560028470418236), 'IC_std': 0.16553718462368272, 'IC_IR': np.float64(-0.6983342441577559), 'IC_positive_ratio': np.float64(0.2063628681418422), 'IC_significant_ratio': np.float64(0.9344151982723545), 'IC_abs_mean': np.float64(0.1622315432818804), 'observation_count': 127804, 'total_periods': 127804}


In [19]:
bsv_simple_ic = cal_ic_mean(bsv_factor,target_rest)
print(f'BSV简单IC均值: {bsv_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    bsv_factor = cal_BSV(E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window = win)
    bsv_ic = cal_rolling_ic_series(bsv_factor,target_rest,min_periods=win)
    print(f'窗口{win}的BSV滚动IC统计: {analyze_ic_series(bsv_ic)}')


BSV简单IC均值: -0.0428
窗口300的BSV滚动IC统计: {'IC_mean': np.float64(-0.05656459002651942), 'IC_std': 0.5817856995703927, 'IC_IR': np.float64(-0.0972258171149417), 'IC_positive_ratio': np.float64(0.4598007483073999), 'IC_significant_ratio': np.float64(0.9824504097873856), 'IC_abs_mean': np.float64(0.5137965307582314), 'observation_count': 134704, 'total_periods': 134704}
窗口600的BSV滚动IC统计: {'IC_mean': np.float64(-0.06643149826342899), 'IC_std': 0.5468366388641106, 'IC_IR': np.float64(-0.12148326125590367), 'IC_positive_ratio': np.float64(0.45705484955804887), 'IC_significant_ratio': np.float64(0.9780661289842564), 'IC_abs_mean': np.float64(0.47821481324238513), 'observation_count': 134404, 'total_periods': 134404}
窗口1200的BSV滚动IC统计: {'IC_mean': np.float64(-0.040524018879750934), 'IC_std': 0.5086562698838331, 'IC_IR': np.float64(-0.07966876902747273), 'IC_positive_ratio': np.float64(0.46264685659621535), 'IC_significant_ratio': np.float64(0.9802547009058025), 'IC_abs_mean': np.float64(0.444043734499

In [23]:
bsv_factor = cal_BSV(E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window=600)
bsv_simple_ic = cal_ic_mean(bsv_factor,target_rest)
print(f'BSV简单IC均值: {bsv_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:    
    bsv_ic = cal_rolling_ic_series(bsv_factor,target_rest,min_periods=win)
    print(f'窗口{win}的BSV滚动IC统计: {analyze_ic_series(bsv_ic)}')

BSV简单IC均值: -0.0428
窗口300的BSV滚动IC统计: {'IC_mean': np.float64(-0.036143444731622135), 'IC_std': 0.6010807842536233, 'IC_IR': np.float64(-0.06013076058736819), 'IC_positive_ratio': np.float64(0.47465554103812807), 'IC_significant_ratio': np.float64(0.9842469414419764), 'IC_abs_mean': np.float64(0.5358956892289333), 'observation_count': 134704, 'total_periods': 134704}
窗口600的BSV滚动IC统计: {'IC_mean': np.float64(-0.06643149826342899), 'IC_std': 0.5468366388641106, 'IC_IR': np.float64(-0.12148326125590367), 'IC_positive_ratio': np.float64(0.45705484955804887), 'IC_significant_ratio': np.float64(0.9780661289842564), 'IC_abs_mean': np.float64(0.47821481324238513), 'observation_count': 134404, 'total_periods': 134404}
窗口1200的BSV滚动IC统计: {'IC_mean': np.float64(-0.11860943531260268), 'IC_std': 0.4821207235290827, 'IC_IR': np.float64(-0.2460160485207765), 'IC_positive_ratio': np.float64(0.40451705479656813), 'IC_significant_ratio': np.float64(0.9775193566709516), 'IC_abs_mean': np.float64(0.43315636073

In [20]:
pastreturn_factor = cal_PastReturn(E_lastprice_rest_all,window = 600)
pastreturn_simple_ic = cal_ic_mean(pastreturn_factor,target_rest)
print(f'PastReturn简单IC均值: {pastreturn_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    pastreturn_factor = cal_PastReturn(E_lastprice_rest_all,window = win)
    pastreturn_ic = cal_rolling_ic_series(pastreturn_factor,target_rest,min_periods=win)
    print(f'窗口{win}的PastReturn滚动IC统计: {analyze_ic_series(pastreturn_ic)}')


PastReturn简单IC均值: -0.1213
窗口300的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.2375689389089519), 'IC_std': 0.5450527101375693, 'IC_IR': np.float64(-0.43586415495299596), 'IC_positive_ratio': np.float64(0.33451616495304554), 'IC_significant_ratio': np.float64(0.9823985746631528), 'IC_abs_mean': np.float64(0.5226877534681732), 'observation_count': 134705, 'total_periods': 134705}
窗口600的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.09155851115196248), 'IC_std': 0.5318954516021859, 'IC_IR': np.float64(-0.1721362927172401), 'IC_positive_ratio': np.float64(0.43900152524087643), 'IC_significant_ratio': np.float64(0.9804471559837804), 'IC_abs_mean': np.float64(0.4693512823999891), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.06031580165545044), 'IC_std': 0.47823376354694125, 'IC_IR': np.float64(-0.12612200612542096), 'IC_positive_ratio': np.float64(0.46210530249243303), 'IC_significant_ratio': np.float64(0.9755315571166997), 'IC_abs_mean'

In [24]:
pastreturn_factor = cal_PastReturn(E_lastprice_rest_all,window = 600)
pastreturn_simple_ic = cal_ic_mean(pastreturn_factor,target_rest)
print(f'PastReturn简单IC均值: {pastreturn_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    pastreturn_ic = cal_rolling_ic_series(pastreturn_factor,target_rest,min_periods=win)
    print(f'窗口{win}的PastReturn滚动IC统计: {analyze_ic_series(pastreturn_ic)}')

PastReturn简单IC均值: -0.1213
窗口300的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.027456153094156426), 'IC_std': 0.6021651678433447, 'IC_IR': np.float64(-0.04559571785344322), 'IC_positive_ratio': np.float64(0.4851341821016295), 'IC_significant_ratio': np.float64(0.9842544820162578), 'IC_abs_mean': np.float64(0.5328056067235729), 'observation_count': 134705, 'total_periods': 134705}
窗口600的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.09155851115196248), 'IC_std': 0.5318954516021859, 'IC_IR': np.float64(-0.1721362927172401), 'IC_positive_ratio': np.float64(0.43900152524087643), 'IC_significant_ratio': np.float64(0.9804471559837804), 'IC_abs_mean': np.float64(0.4693512823999891), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的PastReturn滚动IC统计: {'IC_mean': np.float64(-0.21195178377426588), 'IC_std': 0.43618386957747524, 'IC_IR': np.float64(-0.4859230213615657), 'IC_positive_ratio': np.float64(0.32693098165240464), 'IC_significant_ratio': np.float64(0.977071110945032), 'IC_abs_mean':

In [12]:
hlr_factor = cal_HLR(E_lastprice_rest_all,window = 600)
hlr_ic = calculate_ic_series(hlr_factor,target_rest)
print(analyze_ic_series(hlr_ic))

{'IC_mean': np.float64(-0.11490123419981563), 'IC_std': 0.15259118796723842, 'IC_IR': np.float64(-0.75300045651709), 'IC_positive_ratio': np.float64(0.022885719577699574), 'IC_significant_ratio': np.float64(0.8764956473421004), 'IC_abs_mean': np.float64(0.13656868140377515), 'observation_count': 134975, 'total_periods': 134975}


In [25]:
hlr_factor = cal_HLR(E_lastprice_rest_all,window = 600)
hlr_simple_ic = cal_ic_mean(hlr_factor,target_rest)
print(f'HLR简单IC均值: {hlr_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    hlr_ic = cal_rolling_ic_series(hlr_factor,target_rest,min_periods=win)
    print(f'窗口{win}的HLR滚动IC统计: {analyze_ic_series(hlr_ic)}')

HLR简单IC均值: -0.0657
窗口300的HLR滚动IC统计: {'IC_mean': np.float64(0.09872057814108924), 'IC_std': 0.6159417527879895, 'IC_IR': np.float64(0.16027583402852932), 'IC_positive_ratio': np.float64(0.5660740135852418), 'IC_significant_ratio': np.float64(0.984276752904495), 'IC_abs_mean': np.float64(0.5561094998921225), 'observation_count': 134705, 'total_periods': 134705}
窗口600的HLR滚动IC统计: {'IC_mean': np.float64(0.031564498329972004), 'IC_std': 0.5603017383918525, 'IC_IR': np.float64(0.056334821342098114), 'IC_positive_ratio': np.float64(0.510829210222834), 'IC_significant_ratio': np.float64(0.9817119898813288), 'IC_abs_mean': np.float64(0.4928909075745818), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的HLR滚动IC统计: {'IC_mean': np.float64(-0.03539381091634553), 'IC_std': 0.5090149401425514, 'IC_IR': np.float64(-0.06953393333883946), 'IC_positive_ratio': np.float64(0.4773140017189193), 'IC_significant_ratio': np.float64(0.973984529726094), 'IC_abs_mean': np.float64(0.4403332173109625), '

In [10]:
random_factor = pd.Series(np.random.randn(len(target_rest)),index=target_rest.index)
random_ic = calculate_ic_series(random_factor,target_rest)
print(analyze_ic_series(random_ic))

{'IC_mean': np.float64(-0.0025588853244871406), 'IC_std': 0.004899966990000531, 'IC_IR': np.float64(-0.5222250128845998), 'IC_positive_ratio': np.float64(0.17772920911279866), 'IC_significant_ratio': np.float64(0.0032672717169846266), 'IC_abs_mean': np.float64(0.003202259123265408), 'observation_count': 134975, 'total_periods': 134975}


In [11]:
downlogreturnstd_factor = cal_DownLogreturn_std(E_lastprice_rest_all,window = 600)
downlogreturnstd_ic = calculate_ic_series(downlogreturnstd_factor,target_rest)
print(analyze_ic_series(downlogreturnstd_ic))

{'IC_mean': np.float64(-0.11609899886389814), 'IC_std': 0.1369944806134042, 'IC_IR': np.float64(-0.8474720904379155), 'IC_positive_ratio': np.float64(0.0012225029451207313), 'IC_significant_ratio': np.float64(0.916573435381458), 'IC_abs_mean': np.float64(0.11772556095970274), 'observation_count': 134969, 'total_periods': 134969}


In [26]:
downlogreturnstd_factor = cal_DownLogreturn_std(E_lastprice_rest_all,window = 600)
downlogreturnstd_simple_ic = cal_ic_mean(downlogreturnstd_factor,target_rest)
print(f'DownLogreturnStd简单IC均值: {downlogreturnstd_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    downlogreturnstd_ic = cal_rolling_ic_series(downlogreturnstd_factor,target_rest,min_periods=win)
    print(f'窗口{win}的DownLogreturnStd滚动IC统计: {analyze_ic_series(downlogreturnstd_ic)}')

DownLogreturnStd简单IC均值: -0.0315
窗口300的DownLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.06606285734429364), 'IC_std': 0.5703797200557162, 'IC_IR': np.float64(-0.11582259154978448), 'IC_positive_ratio': np.float64(0.44557866056912077), 'IC_significant_ratio': np.float64(0.9807125516893221), 'IC_abs_mean': np.float64(0.505103546111453), 'observation_count': 134699, 'total_periods': 134699}
窗口600的DownLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.07418583516542172), 'IC_std': 0.535638344675094, 'IC_IR': np.float64(-0.13849985891211944), 'IC_positive_ratio': np.float64(0.445256289109294), 'IC_significant_ratio': np.float64(0.9804983668033245), 'IC_abs_mean': np.float64(0.472841321166631), 'observation_count': 134399, 'total_periods': 134399}
窗口1200的DownLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.10416570048194054), 'IC_std': 0.49105040145655815, 'IC_IR': np.float64(-0.21212832770926018), 'IC_positive_ratio': np.float64(0.4096667389143417), 'IC_significant_ratio': np.float64(0.980478179956

In [12]:
downlogreturn_factor = cal_DownLogreturn(E_lastprice_rest_all,window = 600)
downlogreturn_ic = calculate_ic_series(downlogreturn_factor,target_rest)
print(analyze_ic_series(downlogreturn_ic))

{'IC_mean': np.float64(0.10580664377771039), 'IC_std': 0.1461737084943664, 'IC_IR': np.float64(0.7238418240020792), 'IC_positive_ratio': np.float64(0.8885900570497147), 'IC_significant_ratio': np.float64(0.6436689634733644), 'IC_abs_mean': np.float64(0.10834022329107298), 'observation_count': 134970, 'total_periods': 134970}


In [27]:
downlogreturn_factor = cal_DownLogreturn(E_lastprice_rest_all,window = 600)
downlogreturn_simple_ic = cal_ic_mean(downlogreturn_factor,target_rest)
print(f'DownLogreturn简单IC均值: {downlogreturn_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    downlogreturn_ic = cal_rolling_ic_series(downlogreturn_factor,target_rest,min_periods=win)
    print(f'窗口{win}的DownLogreturn滚动IC统计: {analyze_ic_series(downlogreturn_ic)}')

DownLogreturn简单IC均值: 0.0250
窗口300的DownLogreturn滚动IC统计: {'IC_mean': np.float64(0.01898503875058575), 'IC_std': 0.5935992106207015, 'IC_IR': np.float64(0.03198292452365949), 'IC_positive_ratio': np.float64(0.5205716406829992), 'IC_significant_ratio': np.float64(0.9846250927988122), 'IC_abs_mean': np.float64(0.5257097836697127), 'observation_count': 134700, 'total_periods': 134700}
窗口600的DownLogreturn滚动IC统计: {'IC_mean': np.float64(0.04884841352844506), 'IC_std': 0.5619810113424346, 'IC_IR': np.float64(0.08692182216576712), 'IC_positive_ratio': np.float64(0.5378571428571428), 'IC_significant_ratio': np.float64(0.9823363095238096), 'IC_abs_mean': np.float64(0.4954022028779571), 'observation_count': 134400, 'total_periods': 134400}
窗口1200的DownLogreturn滚动IC统计: {'IC_mean': np.float64(0.10348211057541706), 'IC_std': 0.5003504682357183, 'IC_IR': np.float64(0.20681925399271534), 'IC_positive_ratio': np.float64(0.5916068759342302), 'IC_significant_ratio': np.float64(0.975186846038864), 'IC_abs_mea

In [13]:
uplogreturnstd_factor = cal_UpLogreturn_std(E_lastprice_rest_all,window = 600)
uplogreturnstd_ic = calculate_ic_series(uplogreturnstd_factor,target_rest)
print(analyze_ic_series(uplogreturnstd_ic))

{'IC_mean': np.float64(-0.09051951001998458), 'IC_std': 0.13915654274927852, 'IC_IR': np.float64(-0.6504869137419979), 'IC_positive_ratio': np.float64(0.16823246991879556), 'IC_significant_ratio': np.float64(0.6128934265900066), 'IC_abs_mean': np.float64(0.09597836947002249), 'observation_count': 134968, 'total_periods': 134968}


In [28]:
uplogreturnstd_factor = cal_UpLogreturn_std(E_lastprice_rest_all,window = 600)
uplogreturnstd_simple_ic = cal_ic_mean(uplogreturnstd_factor,target_rest)
print(f'UpLogreturnStd简单IC均值: {uplogreturnstd_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    uplogreturnstd_ic = cal_rolling_ic_series(uplogreturnstd_factor,target_rest,min_periods=win)
    print(f'窗口{win}的UpLogreturnStd滚动IC统计: {analyze_ic_series(uplogreturnstd_ic)}')

UpLogreturnStd简单IC均值: -0.0151
窗口300的UpLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.034208868667104264), 'IC_std': 0.5707660821172624, 'IC_IR': np.float64(-0.059935006194141965), 'IC_positive_ratio': np.float64(0.47132102926546793), 'IC_significant_ratio': np.float64(0.983548382307087), 'IC_abs_mean': np.float64(0.507037894526769), 'observation_count': 134698, 'total_periods': 134698}
窗口600的UpLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.0876119331928442), 'IC_std': 0.5489863384600895, 'IC_IR': np.float64(-0.15958854903128608), 'IC_positive_ratio': np.float64(0.42507328978109793), 'IC_significant_ratio': np.float64(0.9794639801187518), 'IC_abs_mean': np.float64(0.49014096409617514), 'observation_count': 134398, 'total_periods': 134398}
窗口1200的UpLogreturnStd滚动IC统计: {'IC_mean': np.float64(-0.144403586453462), 'IC_std': 0.49063772059190763, 'IC_IR': np.float64(-0.29431815042523196), 'IC_positive_ratio': np.float64(0.3792657588304758), 'IC_significant_ratio': np.float64(0.9809937368271574)

In [14]:
uplogreturn_factor = cal_UpLogreturn(E_lastprice_rest_all,window = 600)
uplogreturn_ic = calculate_ic_series(uplogreturn_factor,target_rest)
print(analyze_ic_series(uplogreturn_ic))

{'IC_mean': np.float64(-0.09362565187949709), 'IC_std': 0.14783473447972964, 'IC_IR': np.float64(-0.6333129504983457), 'IC_positive_ratio': np.float64(0.2366639006608778), 'IC_significant_ratio': np.float64(0.5999985182111846), 'IC_abs_mean': np.float64(0.10151181469380549), 'observation_count': 134972, 'total_periods': 134972}


In [29]:
uplogreturn_factor = cal_UpLogreturn(E_lastprice_rest_all,window = 600)
uplogreturn_simple_ic = cal_ic_mean(uplogreturn_factor,target_rest)
print(f'UpLogreturn简单IC均值: {uplogreturn_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    uplogreturn_ic = cal_rolling_ic_series(uplogreturn_factor,target_rest,min_periods=win)
    print(f'窗口{win}的UpLogreturn滚动IC统计: {analyze_ic_series(uplogreturn_ic)}')

UpLogreturn简单IC均值: -0.0157
窗口300的UpLogreturn滚动IC统计: {'IC_mean': np.float64(-0.050841270593075236), 'IC_std': 0.5834621775682619, 'IC_IR': np.float64(-0.08713721736851929), 'IC_positive_ratio': np.float64(0.46279936452316967), 'IC_significant_ratio': np.float64(0.9838532464254428), 'IC_abs_mean': np.float64(0.5180898477230486), 'observation_count': 134702, 'total_periods': 134702}
窗口600的UpLogreturn滚动IC统计: {'IC_mean': np.float64(-0.09421401992947549), 'IC_std': 0.548877115764515, 'IC_IR': np.float64(-0.17164865727412867), 'IC_positive_ratio': np.float64(0.4249564738619961), 'IC_significant_ratio': np.float64(0.9811535542625853), 'IC_abs_mean': np.float64(0.48956179294073104), 'observation_count': 134402, 'total_periods': 134402}
窗口1200的UpLogreturn滚动IC统计: {'IC_mean': np.float64(-0.15146366798897792), 'IC_std': 0.4847875714665318, 'IC_IR': np.float64(-0.3124330673964782), 'IC_positive_ratio': np.float64(0.3761528228277604), 'IC_significant_ratio': np.float64(0.9774816519932438), 'IC_abs_me

In [10]:
dif_factor = cal_DIF(E_lastprice_rest_all,window_long = 600,window_short=300)
dif_ic = calculate_ic_series(dif_factor,target_rest)
print(analyze_ic_series(dif_ic))

{'IC_mean': np.float64(0.25404924558949166), 'IC_std': 0.13319721261484882, 'IC_IR': np.float64(1.9073165316461755), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.25404924558949166), 'observation_count': 134975, 'total_periods': 134975}


In [30]:
dif_factor = cal_DIF(E_lastprice_rest_all,window_long = 600,window_short=300)
dif_simple_ic = cal_ic_mean(dif_factor,target_rest)
print(f'DIF简单IC均值: {dif_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    dif_ic = cal_rolling_ic_series(dif_factor,target_rest,min_periods=win)
    print(f'窗口{win}的DIF滚动IC统计: {analyze_ic_series(dif_ic)}')

DIF简单IC均值: 0.1908
窗口300的DIF滚动IC统计: {'IC_mean': np.float64(0.3705376546489238), 'IC_std': 0.5814444147000337, 'IC_IR': np.float64(0.6372709846049233), 'IC_positive_ratio': np.float64(0.7375598530121377), 'IC_significant_ratio': np.float64(0.9870234957870903), 'IC_abs_mean': np.float64(0.624656426022817), 'observation_count': 134705, 'total_periods': 134705}
窗口600的DIF滚动IC统计: {'IC_mean': np.float64(0.4424704848765855), 'IC_std': 0.5005404249768203, 'IC_IR': np.float64(0.8839855140512898), 'IC_positive_ratio': np.float64(0.8081321379412968), 'IC_significant_ratio': np.float64(0.9891298686804806), 'IC_abs_mean': np.float64(0.6054870611977954), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的DIF滚动IC统计: {'IC_mean': np.float64(0.4718648111415195), 'IC_std': 0.4013224618820482, 'IC_IR': np.float64(1.1757747346825662), 'IC_positive_ratio': np.float64(0.8661783939314674), 'IC_significant_ratio': np.float64(0.9848510892717014), 'IC_abs_mean': np.float64(0.5566125796320732), 'observati

In [ ]:
dea_factor = cal_DEA(E_lastprice_rest_all,window_long = 600,window_short=300,window_dea=150)
dea_ic = calculate_ic_series(dea_factor,target_rest)
print(analyze_ic_series(dea_ic))

{'IC_mean': np.float64(0.24844639653147377), 'IC_std': 0.1347328100061809, 'IC_IR': np.float64(1.8439932821120277), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.24844639653147377), 'observation_count': 134975, 'total_periods': 134975}


In [32]:
dea_factor = cal_DEA(E_lastprice_rest_all,window_long = 600,window_short=300,window_dea=150)
dea_simple_ic = cal_ic_mean(dea_factor,target_rest)
print(f'DEA简单IC均值: {dea_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    dea_ic = cal_rolling_ic_series(dea_factor,target_rest,min_periods=win)
    print(f'窗口{win}的DEA滚动IC统计: {analyze_ic_series(dea_ic)}')

DEA简单IC均值: 0.1766
窗口300的DEA滚动IC统计: {'IC_mean': np.float64(0.16191817028857333), 'IC_std': 0.6484546190426976, 'IC_IR': np.float64(0.24969853792947042), 'IC_positive_ratio': np.float64(0.5964811996585131), 'IC_significant_ratio': np.float64(0.9868007869047177), 'IC_abs_mean': np.float64(0.6003510444277464), 'observation_count': 134705, 'total_periods': 134705}
窗口600的DEA滚动IC统计: {'IC_mean': np.float64(0.25942932177970507), 'IC_std': 0.5407045030778016, 'IC_IR': np.float64(0.47979870761752463), 'IC_positive_ratio': np.float64(0.6862170306164205), 'IC_significant_ratio': np.float64(0.9861612291209404), 'IC_abs_mean': np.float64(0.5310934092473165), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的DEA滚动IC统计: {'IC_mean': np.float64(0.36372220721497284), 'IC_std': 0.4326092803646398, 'IC_IR': np.float64(0.8407637647264453), 'IC_positive_ratio': np.float64(0.794432196106274), 'IC_significant_ratio': np.float64(0.9812563058181683), 'IC_abs_mean': np.float64(0.4939761761724665), 'obse

In [13]:
vwap_factor = cal_VWAP(E_df_rest_all['TradeSellAmount'],E_df_rest_all['TradeBuyAmount'],
                       E_df_rest_all['TradeSellVolume'],E_df_rest_all['TradeBuyVolume'],window=600)
vwap_ic = calculate_ic_series(vwap_factor,target_rest)
print(analyze_ic_series(vwap_ic))

{'IC_mean': np.float64(-0.16531167626854926), 'IC_std': 0.13663244496093355, 'IC_IR': np.float64(-1.2099005936387641), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.16531167626854926), 'observation_count': 134975, 'total_periods': 134975}


In [33]:
vwap_factor = cal_VWAP(E_df_rest_all['TradeSellAmount'],E_df_rest_all['TradeBuyAmount'],
                       E_df_rest_all['TradeSellVolume'],E_df_rest_all['TradeBuyVolume'],window=600)
vwap_simple_ic = cal_ic_mean(vwap_factor,target_rest)
print(f'VWAP简单IC均值: {vwap_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    vwap_ic = cal_rolling_ic_series(vwap_factor,target_rest,min_periods=win)
    print(f'窗口{win}的VWAP滚动IC统计: {analyze_ic_series(vwap_ic)}')

VWAP简单IC均值: -0.0721
窗口300的VWAP滚动IC统计: {'IC_mean': np.float64(-0.09532561716081676), 'IC_std': 0.6767301389354957, 'IC_IR': np.float64(-0.1408620832386231), 'IC_positive_ratio': np.float64(0.4430867451096841), 'IC_significant_ratio': np.float64(0.9882112764930775), 'IC_abs_mean': np.float64(0.6167581499300352), 'observation_count': 134705, 'total_periods': 134705}
窗口600的VWAP滚动IC统计: {'IC_mean': np.float64(-0.14227259408770862), 'IC_std': 0.6090932554303466, 'IC_IR': np.float64(-0.23358097108986675), 'IC_positive_ratio': np.float64(0.40613816450280865), 'IC_significant_ratio': np.float64(0.9845467058517168), 'IC_abs_mean': np.float64(0.5569370060997167), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的VWAP滚动IC统计: {'IC_mean': np.float64(-0.2720812719493054), 'IC_std': 0.48603445466901524, 'IC_IR': np.float64(-0.5597983215708239), 'IC_positive_ratio': np.float64(0.28995926908560965), 'IC_significant_ratio': np.float64(0.9832592205074548), 'IC_abs_mean': np.float64(0.49131423567

In [14]:
rs_factor = cal_RS(E_lastprice_rest_all,window=600)
rs_ic = calculate_ic_series(rs_factor,target_rest)
print(analyze_ic_series(rs_ic))

{'IC_mean': np.float64(-0.18227422023556875), 'IC_std': 0.1524276785207788, 'IC_IR': np.float64(-1.1958078874154165), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.18227422023556875), 'observation_count': 134975, 'total_periods': 134975}


In [39]:
rs_factor = cal_RS(E_lastprice_rest_all,window=600)
rs_simple_ic = cal_ic_mean(rs_factor,target_rest)  
print(f'RS简单IC均值: {rs_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    rs_ic = cal_rolling_ic_series(rs_factor,target_rest,min_periods=win)
    print(f'窗口{win}的RS滚动IC统计: {analyze_ic_series(rs_ic)}')

RS简单IC均值: -0.1159
窗口300的RS滚动IC统计: {'IC_mean': np.float64(-0.48337227570975844), 'IC_std': 0.47560333389896986, 'IC_IR': np.float64(-1.0163349187380568), 'IC_positive_ratio': np.float64(0.1754277866448907), 'IC_significant_ratio': np.float64(0.9893619390520025), 'IC_abs_mean': np.float64(0.6165502458952903), 'observation_count': 134705, 'total_periods': 134705}
窗口600的RS滚动IC统计: {'IC_mean': np.float64(-0.4979859756770195), 'IC_std': 0.4331781042747925, 'IC_IR': np.float64(-1.1496102198210723), 'IC_positive_ratio': np.float64(0.15053011420706075), 'IC_significant_ratio': np.float64(0.9883486477437595), 'IC_abs_mean': np.float64(0.6007607213701973), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的RS滚动IC统计: {'IC_mean': np.float64(-0.4462110750304161), 'IC_std': 0.3660786379714633, 'IC_IR': np.float64(-1.218894053755738), 'IC_positive_ratio': np.float64(0.1267964575314824), 'IC_significant_ratio': np.float64(0.9878704084301783), 'IC_abs_mean': np.float64(0.5255803827810158), 'obs

In [40]:
ewp_factor = cal_EWP(E_lastprice_rest_all,window=600)
ewp_simple_ic = cal_ic_mean(ewp_factor,target_rest)
print(f'EWP简单IC均值: {ewp_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    ewp_ic = cal_rolling_ic_series(ewp_factor,target_rest,min_periods=win)
    print(f'窗口{win}的EWP滚动IC统计: {analyze_ic_series(ewp_ic)}')

EWP简单IC均值: -0.0760
窗口300的EWP滚动IC统计: {'IC_mean': np.float64(-0.22437828194089307), 'IC_std': 0.6376134142726103, 'IC_IR': np.float64(-0.35190332718589984), 'IC_positive_ratio': np.float64(0.3621691845143091), 'IC_significant_ratio': np.float64(0.987194239263576), 'IC_abs_mean': np.float64(0.6079129103536767), 'observation_count': 134705, 'total_periods': 134705}
窗口600的EWP滚动IC统计: {'IC_mean': np.float64(-0.2987414512206192), 'IC_std': 0.5422242140349214, 'IC_IR': np.float64(-0.5509555705702569), 'IC_positive_ratio': np.float64(0.2946467765336111), 'IC_significant_ratio': np.float64(0.9846880696402663), 'IC_abs_mean': np.float64(0.5497630812040079), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的EWP滚动IC统计: {'IC_mean': np.float64(-0.4061813847140232), 'IC_std': 0.4259321672391378, 'IC_IR': np.float64(-0.9536292770439534), 'IC_positive_ratio': np.float64(0.1988341242853406), 'IC_significant_ratio': np.float64(0.9841411008557229), 'IC_abs_mean': np.float64(0.5235511518772785), '

In [41]:
ewp_reciprocalmomentum_factor = cal_EWP_reciprocalmomentum(E_lastprice_rest_all,window=600)
ewp_reciprocalmomentum_simple_ic = cal_ic_mean(ewp_reciprocalmomentum_factor,target_rest)
print(f'EWP Reciprocal Momentum简单IC均值: {ewp_reciprocalmomentum_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    ewp_reciprocalmomentum_ic = cal_rolling_ic_series(ewp_reciprocalmomentum_factor,target_rest,min_periods=win)
    print(f'窗口{win}的EWP Reciprocal Momentum滚动IC统计: {analyze_ic_series(ewp_reciprocalmomentum_ic)}')

EWP Reciprocal Momentum简单IC均值: 0.1611
窗口300的EWP Reciprocal Momentum滚动IC统计: {'IC_mean': np.float64(0.1594063351825099), 'IC_std': 0.6668850630349928, 'IC_IR': np.float64(0.23903119745560342), 'IC_positive_ratio': np.float64(0.6001113544411864), 'IC_significant_ratio': np.float64(0.9867859396458929), 'IC_abs_mean': np.float64(0.6197744018266477), 'observation_count': 134705, 'total_periods': 134705}
窗口600的EWP Reciprocal Momentum滚动IC统计: {'IC_mean': np.float64(0.22037145772009298), 'IC_std': 0.5805051434037763, 'IC_IR': np.float64(0.3796201639626323), 'IC_positive_ratio': np.float64(0.6519400319928574), 'IC_significant_ratio': np.float64(0.9827908187939437), 'IC_abs_mean': np.float64(0.5486829068455074), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的EWP Reciprocal Momentum滚动IC统计: {'IC_mean': np.float64(0.3195240322012288), 'IC_std': 0.4785306016380505, 'IC_IR': np.float64(0.6677191199632191), 'IC_positive_ratio': np.float64(0.7418407383879526), 'IC_significant_ratio': np.flo

In [17]:
rsimomentum_factor = cal_RSImomentum(E_lastprice_rest_all,window=600)
rsimomentum_ic = calculate_ic_series(rsimomentum_factor,target_rest)
print(analyze_ic_series(rsimomentum_ic))

{'IC_mean': np.float64(-0.2345901123578258), 'IC_std': 0.1441830769833014, 'IC_IR': np.float64(-1.6270294494061526), 'IC_positive_ratio': np.float64(0.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.2345901123578258), 'observation_count': 134975, 'total_periods': 134975}


In [42]:
rsimomentum_factor = cal_RSImomentum(E_lastprice_rest_all,window=600)
rsimomentum_simple_ic = cal_ic_mean(rsimomentum_factor,target_rest)
print(f'RSI Momentum简单IC均值: {rsimomentum_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    rsimomentum_ic = cal_rolling_ic_series(rsimomentum_factor,target_rest,min_periods=win)
    print(f'窗口{win}的RSI Momentum滚动IC统计: {analyze_ic_series(rsimomentum_ic)}')

RSI Momentum简单IC均值: -0.0480
窗口300的RSI Momentum滚动IC统计: {'IC_mean': np.float64(-0.2788182147126568), 'IC_std': 0.5505240121031418, 'IC_IR': np.float64(-0.5064596794742889), 'IC_positive_ratio': np.float64(0.31280947255113023), 'IC_significant_ratio': np.float64(0.9850042685869121), 'IC_abs_mean': np.float64(0.5481747153721258), 'observation_count': 134705, 'total_periods': 134705}
窗口600的RSI Momentum滚动IC统计: {'IC_mean': np.float64(-0.29118655929591974), 'IC_std': 0.5068696282959289, 'IC_IR': np.float64(-0.5744801878835685), 'IC_positive_ratio': np.float64(0.2868568877645921), 'IC_significant_ratio': np.float64(0.98355715933187), 'IC_abs_mean': np.float64(0.5131720347234912), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的RSI Momentum滚动IC统计: {'IC_mean': np.float64(-0.23645208565045053), 'IC_std': 0.42920818727637483, 'IC_IR': np.float64(-0.5509030178359454), 'IC_positive_ratio': np.float64(0.2902731587010949), 'IC_significant_ratio': np.float64(0.9788647658906617), 'IC_abs_mea

In [9]:
obi_factor = cal_OBI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
obi_ic = calculate_ic_series(obi_factor,target_rest)
print(analyze_ic_series(obi_ic))

{'IC_mean': np.float64(-0.15996495077893572), 'IC_std': 0.148719490380236, 'IC_IR': np.float64(-1.0756152429647794), 'IC_positive_ratio': np.float64(0.0004000740877940359), 'IC_significant_ratio': np.float64(0.9995184293387664), 'IC_abs_mean': np.float64(0.16007054730776862), 'observation_count': 134975, 'total_periods': 134975}


In [43]:
obi_factor = cal_OBI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
obi_simple_ic = cal_ic_mean(obi_factor,target_rest)
print(f'OBI简单IC均值: {obi_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    obi_ic = cal_rolling_ic_series(obi_factor,target_rest,min_periods=win)
    print(f'窗口{win}的OBI滚动IC统计: {analyze_ic_series(obi_ic)}')

OBI简单IC均值: -0.0654
窗口300的OBI滚动IC统计: {'IC_mean': np.float64(-0.29733788299764385), 'IC_std': 0.5632113303324761, 'IC_IR': np.float64(-0.5279330634597119), 'IC_positive_ratio': np.float64(0.30358190119149253), 'IC_significant_ratio': np.float64(0.9857689024163914), 'IC_abs_mean': np.float64(0.5691648336479561), 'observation_count': 134705, 'total_periods': 134705}
窗口600的OBI滚动IC统计: {'IC_mean': np.float64(-0.3293540256048878), 'IC_std': 0.5151111294732098, 'IC_IR': np.float64(-0.6393844100043212), 'IC_positive_ratio': np.float64(0.26842007365797405), 'IC_significant_ratio': np.float64(0.9851865629998884), 'IC_abs_mean': np.float64(0.5435935122629104), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的OBI滚动IC统计: {'IC_mean': np.float64(-0.31779997713626934), 'IC_std': 0.4315835843164152, 'IC_IR': np.float64(-0.7363578891436114), 'IC_positive_ratio': np.float64(0.24222562684503568), 'IC_significant_ratio': np.float64(0.9771308994432196), 'IC_abs_mean': np.float64(0.4634212700364085

In [11]:
oir_factor = cal_OIR(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
oir_ic = calculate_ic_series(oir_factor,target_rest)
print(analyze_ic_series(oir_ic))

{'IC_mean': np.float64(-0.1425737449361117), 'IC_std': 0.15913563408869916, 'IC_IR': np.float64(-0.895925954941329), 'IC_positive_ratio': np.float64(0.0012224486015928876), 'IC_significant_ratio': np.float64(0.9998147805149101), 'IC_abs_mean': np.float64(0.14280645110141013), 'observation_count': 134975, 'total_periods': 134975}


In [44]:
oir_factor = cal_OIR(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
oir_simple_ic = cal_ic_mean(oir_factor,target_rest)
print(f'OIR简单IC均值: {oir_simple_ic:.4f}')
for win in [300,600,1200,2400,4800]:
    oir_ic = cal_rolling_ic_series(oir_factor,target_rest,min_periods=win)
    print(f'窗口{win}的OIR滚动IC统计: {analyze_ic_series(oir_ic)}')

OIR简单IC均值: -0.0532
窗口300的OIR滚动IC统计: {'IC_mean': np.float64(-0.2746516331928684), 'IC_std': 0.5646143711373814, 'IC_IR': np.float64(-0.486441095432267), 'IC_positive_ratio': np.float64(0.31590512601610926), 'IC_significant_ratio': np.float64(0.9844103782339185), 'IC_abs_mean': np.float64(0.5584020715802613), 'observation_count': 134705, 'total_periods': 134705}
窗口600的OIR滚动IC统计: {'IC_mean': np.float64(-0.3130895933060135), 'IC_std': 0.5180297836892869, 'IC_IR': np.float64(-0.6043853136710841), 'IC_positive_ratio': np.float64(0.2727205089096388), 'IC_significant_ratio': np.float64(0.9847699118336372), 'IC_abs_mean': np.float64(0.5387387204945379), 'observation_count': 134405, 'total_periods': 134405}
窗口1200的OIR滚动IC统计: {'IC_mean': np.float64(-0.31340392745175283), 'IC_std': 0.43528062586052896, 'IC_IR': np.float64(-0.7200043117751181), 'IC_positive_ratio': np.float64(0.23796569634916484), 'IC_significant_ratio': np.float64(0.9806883150853855), 'IC_abs_mean': np.float64(0.46615771416685103)

In [23]:
voi_factor = cal_VOI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],window=600)
voi_ic = calculate_ic_series(voi_factor,target_rest)
print(analyze_ic_series(voi_ic))

{'IC_mean': np.float64(-0.15996495077893572), 'IC_std': 0.148719490380236, 'IC_IR': np.float64(-1.0756152429647794), 'IC_positive_ratio': np.float64(0.0004000740877940359), 'IC_significant_ratio': np.float64(0.9995184293387664), 'IC_abs_mean': np.float64(0.16007054730776862), 'observation_count': 134975, 'total_periods': 134975}


In [13]:
avo_factor = cal_AVO(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],E_df_rest_all['OrderBuyNum'],E_df_rest_all['OrderSellNum'],window=600)
avo_ic = calculate_ic_series(avo_factor,target_rest)
print(analyze_ic_series(avo_ic))

{'IC_mean': np.float64(-0.13279016098832486), 'IC_std': 0.11750694008589367, 'IC_IR': np.float64(-1.130062283055449), 'IC_positive_ratio': np.float64(3.704389701796629e-05), 'IC_significant_ratio': np.float64(0.9999851824411928), 'IC_abs_mean': np.float64(0.13279463401141117), 'observation_count': 134975, 'total_periods': 134975}


In [15]:
bsvmultiobi_factor = cal_BSVmultiOBI(E_df_rest_all['OrderBuyVolume'],E_df_rest_all['OrderSellVolume'],E_df_rest_all['BidPrice1'],E_df_rest_all['AskPrice1'],window=600)
bsvmultiobi_ic = calculate_ic_series(bsvmultiobi_factor,target_rest)
print(analyze_ic_series(bsvmultiobi_ic))

{'IC_mean': np.float64(-0.16006856641187164), 'IC_std': 0.14417031302614142, 'IC_IR': np.float64(-1.1102741129711462), 'IC_positive_ratio': np.float64(0.0002815357031724628), 'IC_significant_ratio': np.float64(0.9999925911657059), 'IC_abs_mean': np.float64(0.16030698401788906), 'observation_count': 134974, 'total_periods': 134974}


In [19]:
updownlogreturn_factor = cal_UpDownLogreturn(E_lastprice_rest_all,window=600)
updownlogreturn_ic = calculate_ic_series(updownlogreturn_factor,target_rest)
print(analyze_ic_series(updownlogreturn_ic))

{'IC_mean': np.float64(0.12913081795369794), 'IC_std': 0.1041451852372689, 'IC_IR': np.float64(1.2399115490505441), 'IC_positive_ratio': np.float64(1.0), 'IC_significant_ratio': np.float64(1.0), 'IC_abs_mean': np.float64(0.12913081795369794), 'observation_count': 134970, 'total_periods': 134970}
